In [29]:
# Import necessary libraries and extensions
%load_ext sql
import csv, sqlite3
import pandas as pd

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [30]:
# Connect to SQLite database
con = sqlite3.connect("HotelReservationAnalysis.db")
cur = con.cursor()

In [31]:
# Load SQL extension
%sql sqlite:///HotelReservationAnalysis.db

In [32]:
# Load dataset into pandas DataFrame
df = pd.read_csv('HotelReservationDataset.csv')
df.head(5)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,room_type_reserved,lead_time,arrival_date,market_segment_type,avg_price_per_room,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,Room_Type 1,224,02-10-2017,Offline,65.00,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,Room_Type 1,5,06-11-2018,Online,106.68,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,Room_Type 1,1,28-02-2018,Online,60.00,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,Room_Type 1,211,20-05-2018,Online,100.00,Canceled
4,INN00005,2,0,1,1,Not Selected,Room_Type 1,48,11-04-2018,Online,94.50,Canceled


In [33]:
# Convert DataFrame to SQL database
df.to_sql("hotel_reservation_data", con, if_exists='replace', index=False,method="multi")

700

In [34]:
# Verify the data loaded correctly
%sql SELECT * FROM hotel_reservation_data limit 5;

 * sqlite:///HotelReservationAnalysis.db
Done.


Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,room_type_reserved,lead_time,arrival_date,market_segment_type,avg_price_per_room,booking_status
INN00001,2,0,1,2,Meal Plan 1,Room_Type 1,224,02-10-2017,Offline,65.0,Not_Canceled
INN00002,2,0,2,3,Not Selected,Room_Type 1,5,06-11-2018,Online,106.68,Not_Canceled
INN00003,1,0,2,1,Meal Plan 1,Room_Type 1,1,28-02-2018,Online,60.0,Canceled
INN00004,2,0,0,2,Meal Plan 1,Room_Type 1,211,20-05-2018,Online,100.0,Canceled
INN00005,2,0,1,1,Not Selected,Room_Type 1,48,11-04-2018,Online,94.5,Canceled


1. What is the total number of reservations in the dataset?

In [35]:
%sql SELECT COUNT(Booking_ID) FROM hotel_reservation_data AS Total_Number_of_Reservations

 * sqlite:///HotelReservationAnalysis.db
Done.


COUNT(Booking_ID)
700


2. Which meal plan is the most popular among guests?

In [36]:
%sql SELECT type_of_meal_plan, COUNT(*) AS count FROM hotel_reservation_data GROUP BY type_of_meal_plan ORDER BY count DESC LIMIT 1

 * sqlite:///HotelReservationAnalysis.db
Done.


type_of_meal_plan,count
Meal Plan 1,527


3. What is the average price per room for reservations involving children?

In [37]:
%sql SELECT ROUND(AVG(avg_price_per_room),2) FROM hotel_reservation_data WHERE no_of_children > 0

 * sqlite:///HotelReservationAnalysis.db
Done.


"ROUND(AVG(avg_price_per_room),2)"
144.57


4. How many reservations were made for the year 20XX (replace XX with the desired year)?

In [38]:
# Convert arrival_date to datetime and extract year
df['arrival_date'] = pd.to_datetime(df['arrival_date'], format='%d-%m-%Y')
df['year'] = df['arrival_date'].dt.year
df.head(2)

,Booking_ID,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,room_type_reserved,lead_time,arrival_date,market_segment_type,avg_price_per_room,booking_status,year
0,INN00001,2,0,1,2,Meal Plan 1,Room_Type 1,224,2017-10-02,Offline,65.00,Not_Canceled,2017
1,INN00002,2,0,2,3,Not Selected,Room_Type 1,5,2018-11-06,Online,106.68,Not_Canceled,2018


In [39]:
# Ensure the DataFrame includes the year column and update the SQL table
df.to_sql("hotel_reservation_data", con, if_exists='replace', index=False,method="multi")

700

In [40]:
%sql SELECT year, COUNT(*) as Reservations FROM hotel_reservation_data GROUP BY year

 * sqlite:///HotelReservationAnalysis.db
Done.


year,Reservations
2017,123
2018,577


In [41]:
%sql SELECT COUNT(*) as Reservations FROM hotel_reservation_data WHERE year = 2018

 * sqlite:///HotelReservationAnalysis.db
Done.


Reservations
577


5. What is the most commonly booked room type?

In [42]:
%sql SELECT room_type_reserved, COUNT(room_type_reserved) AS count FROM hotel_reservation_data GROUP BY room_type_reserved ORDER BY count DESC LIMIT 1

 * sqlite:///HotelReservationAnalysis.db
Done.


room_type_reserved,count
Room_Type 1,534


6. How many reservations fall on a weekend (no_of_weekend_nights > 0)?

In [43]:
%sql SELECT COUNT(*) AS ReservationsOnWeekend FROM hotel_reservation_data WHERE no_of_weekend_nights > 0

 * sqlite:///HotelReservationAnalysis.db
Done.


ReservationsOnWeekend
383


7. What is the highest and lowest lead time for reservations?

In [44]:
%sql SELECT MAX(lead_time) AS HighestLeadTime, MIN(lead_time) AS LowestLeadTime FROM hotel_reservation_data

 * sqlite:///HotelReservationAnalysis.db
Done.


HighestLeadTime,LowestLeadTime
443,0


8. What is the most common market segment type for reservations?

In [45]:
%sql SELECT market_segment_type, COUNT(market_segment_type) AS Count FROM hotel_reservation_data GROUP BY market_segment_type ORDER BY Count DESC LIMIT 1

 * sqlite:///HotelReservationAnalysis.db
Done.


market_segment_type,Count
Online,518


9. How many reservations have a booking status of "Confirmed"?

In [46]:
%sql SELECT COUNT(*) FROM hotel_reservation_data WHERE booking_status = 'Not_Canceled'

 * sqlite:///HotelReservationAnalysis.db
Done.


COUNT(*)
493


10. What is the total number of adults and children across all reservations?

In [47]:
%sql SELECT SUM(no_of_adults), SUM(no_of_children) FROM hotel_reservation_data

 * sqlite:///HotelReservationAnalysis.db
Done.


SUM(no_of_adults),SUM(no_of_children)
1316,69


11. What is the average number of weekend nights for reservations involving children?

In [48]:
%sql SELECT AVG(no_of_weekend_nights) FROM hotel_reservation_data WHERE no_of_children >0

 * sqlite:///HotelReservationAnalysis.db
Done.


AVG(no_of_weekend_nights)
1.0


12. How many reservations were made in each month of the year?

In [49]:
%sql SELECT strftime('%m', arrival_date) AS month, COUNT(*) FROM hotel_reservation_data GROUP BY month ORDER BY month

 * sqlite:///HotelReservationAnalysis.db
Done.


month,COUNT(*)
01,11
02,28
03,52
04,67
05,55
06,84
07,44
08,70
09,80
10,103


13. What is the average number of nights (both weekend and weekday) spent by guests for each room type?

In [50]:
%sql SELECT room_type_reserved, ROUND(AVG(no_of_weekend_nights + no_of_week_nights),2) AS avg_nights FROM hotel_reservation_data GROUP BY room_type_reserved

 * sqlite:///HotelReservationAnalysis.db
Done.


room_type_reserved,avg_nights
Room_Type 1,2.88
Room_Type 2,3.0
Room_Type 4,3.8
Room_Type 5,2.5
Room_Type 6,3.61
Room_Type 7,2.67


14. For reservations involving children, what is the most common room type, and what is the average price for that room type?

In [51]:
%sql SELECT room_type_reserved, COUNT(room_type_reserved) AS Count, AVG(avg_price_per_room) AS AvgPrice FROM hotel_reservation_data WHERE no_of_children > 0 GROUP BY room_type_reserved ORDER BY Count DESC LIMIT 1

 * sqlite:///HotelReservationAnalysis.db
Done.


room_type_reserved,Count,AvgPrice
Room_Type 1,24,123.12291666666665


15. Find the market segment type that generates the highest average price per room.

In [52]:
%sql SELECT market_segment_type, ROUND(AVG(avg_price_per_room),2) AS AvgPrice FROM hotel_reservation_data GROUP BY market_segment_type ORDER BY AvgPrice DESC LIMIT 1

 * sqlite:///HotelReservationAnalysis.db
Done.


market_segment_type,AvgPrice
Online,112.46
